<a href="https://colab.research.google.com/github/shriyan44/Lab-6-DATA-301/blob/main/Copy_of_DATA_301_Lab_6B_Shriya_Nimmagadda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [1]:

# AjDIyFMNcKFfc6oWn8OmF_gxlT8rUMkepcqE6YB2z2EsGyyRFQMSAMXVoKNFH8Lq
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df_stations = pd.read_csv("/content/drive/My Drive/stations.csv")
df_bay = df_stations[df_stations["networkid"] == "bay-wheels"]
# df_bay.reset_index(drop=True)
df_bay


Mounted at /content/drive


,Unnamed: 0,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,extra.ebikes,extra.has_ebikes,extra.bike_uids,extra.number,extra.slots,networkname,networkid
6230,6230,11.0,4,d0e8f4f1834b7b33a3faf8882f567ab8,37.849735,-122.270582,Harmon St at Adeline St,2022-02-25T22:07:54.389000Z,NaN,1.645808e+09,1.0,1.0,340,0.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6231,6231,3.0,8,983514094dd808b1604da2dcfc2d09af,37.336188,-121.889277,Fountain Alley at S 2nd St,2022-02-25T22:07:54.390000Z,NaN,1.645819e+09,1.0,1.0,341,0.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6232,6232,3.0,20,da17603652106fda93da4e255a5b0a22,37.322125,-121.881090,Oak St at 1st St,2022-02-25T22:07:54.396000Z,NaN,1.645809e+09,1.0,1.0,298,0.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6233,6233,1.0,14,7a21c92b3b4cd2f7759107b4fdebf869,37.323678,-121.874119,Bestor Art Park,2022-02-25T22:07:54.420000Z,NaN,1.645825e+09,1.0,1.0,299,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6234,6234,22.0,4,ce34d38fb230a23c1ced12d1e16df294,37.325998,-121.877120,5th St at Virginia St,2022-02-25T22:07:54.421000Z,NaN,1.645818e+09,1.0,1.0,296,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6707,6707,18.0,5,df9c9c79952699cc041118a331ea9cc2,37.724893,-122.443238,Balboa Park (San Jose Ave at Sgt. John V. Youn...,2022-02-25T22:07:55.424000Z,NaN,1.645826e+09,1.0,1.0,501,4.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6708,6708,17.0,10,5dc0d98ea835b4c9743d4e2cdd515dde,37.726266,-122.479462,SFSU University Park North - West,2022-02-25T22:07:55.739000Z,NaN,1.645808e+09,1.0,1.0,566,0.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6709,6709,9.0,6,97871d03b345e3aa1de1dbed3640b700,37.725603,-122.476561,SFSU University Park North - East,2022-02-25T22:07:55.743000Z,NaN,1.645823e+09,1.0,1.0,567,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6710,6710,6.0,13,e427fa5d07c952f21297dbd9c7693407,37.723426,-122.455102,Brighton Ave at Ocean Ave,2022-02-25T22:07:55.738000Z,NaN,1.645818e+09,1.0,1.0,565,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels


In [2]:
df_bay1 = df_bay.loc[[6232,6234]]
df_bay1

,Unnamed: 0,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,extra.ebikes,extra.has_ebikes,extra.bike_uids,extra.number,extra.slots,networkname,networkid
6232,6232,3.0,20,da17603652106fda93da4e255a5b0a22,37.322125,-121.88109,Oak St at 1st St,2022-02-25T22:07:54.396000Z,NaN,1.645809e+09,1.0,1.0,298,0.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels
6234,6234,22.0,4,ce34d38fb230a23c1ced12d1e16df294,37.325998,-121.87712,5th St at Virginia St,2022-02-25T22:07:54.421000Z,NaN,1.645818e+09,1.0,1.0,296,1.0,True,NaN,NaN,NaN,Bay Wheels,bay-wheels


In [3]:
import requests
import time
import pandas as pd
locinfo = []

for i in df_bay.index:
  response = requests.get("http://dev.virtualearth.net/REST/v1/Locations/" + str(df_bay["latitude"][i]) 
  + "," + str(df_bay["longitude"][i]) + "?includeEntityTypes=address&key=AjDIyFMNcKFfc6oWn8OmF_gxlT8rUMkepcqE6YB2z2EsGyyRFQMSAMXVoKNFH8Lq")
  locinfo.append(response.json()["resourceSets"][0])
  # print(locinfo)
  # print(i)
  time.sleep(0.2)



# df_sf = df_all[df_all["San Francisco"]]
# df_sf





In [9]:
df_all = pd.json_normalize(locinfo, "resources", errors="ignore")
df_all["stationName"] = df_bay["name"].to_numpy()
df_all["latitude"] = df_bay["latitude"].to_numpy()
df_all["longitude"] = df_bay["longitude"].to_numpy()
df_all.head()


,__type,bbox,name,confidence,entityType,geocodePoints,matchCodes,point.type,point.coordinates,address.addressLine,address.adminDistrict,address.adminDistrict2,address.countryRegion,address.formattedAddress,address.intersection.baseStreet,address.intersection.secondaryStreet1,address.intersection.intersectionType,address.intersection.displayName,address.locality,address.postalCode,address.intersection.secondaryStreet2,stationName,latitude,longitude
0,Location:http://schemas.microsoft.com/search/l...,"[37.845923782429324, -122.27708843265201, 37.8...","1881 Harmon St, Berkeley, CA 94703, United States",High,Address,"[{'type': 'Point', 'coordinates': [37.8497865,...",[Good],Point,"[37.8497865, -122.2705663]",1881 Harmon St,CA,Alameda Co.,United States,"1881 Harmon St, Berkeley, CA 94703, United States",Harmon St,Adeline St,Near,Harmon St and Adeline St,Berkeley,94703,NaN,Harmon St at Adeline St,37.849735,-122.270582
1,Location:http://schemas.microsoft.com/search/l...,"[37.332329782429326, -121.89561398159309, 37.3...","10 S 2nd St, San Jose, CA 95113, United States",High,Address,"[{'type': 'Point', 'coordinates': [37.3361925,...",[Good],Point,"[37.3361925, -121.8891367]",10 S 2nd St,CA,Santa Clara Co.,United States,"10 S 2nd St, San Jose, CA 95113, United States",S 2nd St,Fountain Alley,Near,S 2nd St and Fountain Alley,Downtown San Jose,95113,NaN,Fountain Alley at S 2nd St,37.336188,-121.889277
2,Location:http://schemas.microsoft.com/search/l...,"[37.318440582429325, -121.88749158445303, 37.3...","35 Oak St, San Jose, CA 95110, United States",High,Address,"[{'type': 'Point', 'coordinates': [37.3223033,...",[Good],Point,"[37.3223033, -121.8810155]",35 Oak St,CA,Santa Clara Co.,United States,"35 Oak St, San Jose, CA 95110, United States",Oak St,State St,Between,"Oak St, between State St and Martha St",Washington Guadalupe,95110,Martha St,Oak St at 1st St,37.322125,-121.881090
3,Location:http://schemas.microsoft.com/search/l...,"[37.319710082429324, -121.88083489383979, 37.3...","271 Bestor St, San Jose, CA 95112, United States",High,Address,"[{'type': 'Point', 'coordinates': [37.3235728,...",[Good],Point,"[37.3235728, -121.8743587]",271 Bestor St,CA,Santa Clara Co.,United States,"271 Bestor St, San Jose, CA 95112, United States",Bestor St,S 6th St,Near,Bestor St and S 6th St,Fairgrounds,95112,NaN,Bestor Art Park,37.323678,-121.874119
4,Location:http://schemas.microsoft.com/search/l...,"[37.32190058242932, -121.88378638260089, 37.32...","826A S 5th St, San Jose, CA 95112, United States",High,Address,"[{'type': 'Point', 'coordinates': [37.3257633,...",[Good],Point,"[37.3257633, -121.87731]",826A S 5th St,CA,Santa Clara Co.,United States,"826A S 5th St, San Jose, CA 95112, United States",S 5th St,E Virginia St,Near,S 5th St and E Virginia St,Fairgrounds,95112,NaN,5th St at Virginia St,37.325998,-121.877120


In [10]:
df_sf = df_all[df_all["address.adminDistrict2"] == "San Francisco Co."]
df_sf


,__type,bbox,name,confidence,entityType,geocodePoints,matchCodes,point.type,point.coordinates,address.addressLine,address.adminDistrict,address.adminDistrict2,address.countryRegion,address.formattedAddress,address.intersection.baseStreet,address.intersection.secondaryStreet1,address.intersection.intersectionType,address.intersection.displayName,address.locality,address.postalCode,address.intersection.secondaryStreet2,stationName,latitude,longitude
25,Location:http://schemas.microsoft.com/search/l...,"[37.75929228242932, -122.4331434870546, 37.767...","392-398 Dolores St, San Francisco, CA 94114, U...",High,Address,"[{'type': 'Point', 'coordinates': [37.763155, ...",[Good],Point,"[37.763155, -122.426629]",392-398 Dolores St,CA,San Francisco Co.,United States,"392-398 Dolores St, San Francisco, CA 94114, U...",Dolores St,Dolores Terrace,Between,"Dolores St, between Dolores Terrace and Chula Ln",Western Addition,94114,Chula Ln,17th St at Dolores St,37.763015,-122.426497
26,Location:http://schemas.microsoft.com/search/l...,"[37.76318188242932, -122.3973483296247, 37.770...","1783 4th St, San Francisco, CA 94158, United S...",Medium,Address,"[{'type': 'Point', 'coordinates': [37.7670446,...",[Good],Point,"[37.7670446, -122.3908335]",1783 4th St,CA,San Francisco Co.,United States,"1783 4th St, San Francisco, CA 94158, United S...",4th St,16th St,Near,4th St and 16th St,Mission Bay,94158,NaN,4th St at 16th St,37.767045,-122.390833
27,Location:http://schemas.microsoft.com/search/l...,"[37.77510228242932, -122.44361417990581, 37.78...","1604 Golden Gate Ave, San Francisco, CA 94115,...",High,Address,"[{'type': 'Point', 'coordinates': [37.778965, ...",[Good],Point,"[37.778965, -122.4370983]",1604 Golden Gate Ave,CA,San Francisco Co.,United States,"1604 Golden Gate Ave, San Francisco, CA 94115,...",Golden Gate Ave,Scott St,Near,Golden Gate Ave and Scott St,Western Addition,94115,NaN,Scott St at Golden Gate Ave,37.778999,-122.436861
28,Location:http://schemas.microsoft.com/search/l...,"[37.779967282429325, -122.40538630872898, 37.7...","694 Folsom St, San Francisco, CA 94107, United...",Medium,Address,"[{'type': 'Point', 'coordinates': [37.78383, -...",[Good],Point,"[37.78383, -122.39887]",694 Folsom St,CA,San Francisco Co.,United States,"694 Folsom St, San Francisco, CA 94107, United...",Folsom St,3rd St,Near,Folsom St and 3rd St,South of Market,94107,NaN,Folsom St at 3rd St,37.783830,-122.398870
30,Location:http://schemas.microsoft.com/search/l...,"[37.72794628242932, -122.45749572871688, 37.73...","696-698 Monterey Blvd, San Francisco, CA 94127...",High,Address,"[{'type': 'Point', 'coordinates': [37.731809, ...",[Good],Point,"[37.731809, -122.450984]",696-698 Monterey Blvd,CA,San Francisco Co.,United States,"696-698 Monterey Blvd, San Francisco, CA 94127...",Monterey Blvd,Gennessee St,Near,Monterey Blvd and Gennessee St,Merced Heights,94127,NaN,Gennessee St at Monterey Blvd,37.731657,-122.451122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,Location:http://schemas.microsoft.com/search/l...,"[37.721032782429326, -122.44947592092983, 37.7...","2007 San Jose Ave, San Francisco, CA 94112, Un...",High,Address,"[{'type': 'Point', 'coordinates': [37.7248955,...",[Good],Point,"[37.7248955, -122.4429648]",2007 San Jose Ave,CA,San Francisco Co.,United States,"2007 San Jose Ave, San Francisco, CA 94112, Un...",NaN,NaN,NaN,NaN,Merced Heights,94112,NaN,Balboa Park (San Jose Ave at Sgt. John V. Youn...,37.724893,-122.443238
478,Location:http://schemas.microsoft.com/search/l...,"[37.72247578242932, -122.4857484477712, 37.730...","220 Buckingham Way, San Francisco, CA 94132, U...",High,Address,"[{'type': 'Point', 'coordinates': [37.7263385,...",[Good],Point,"[37.7263385, -122.4792372]",220 Buckingham Way,CA,San Francisco Co.,United States,"220 Buckingham Way, San Francisco, CA 94132, U...",NaN,NaN,NaN,NaN,Merced Heights,94132,NaN,SFSU University Park North - West,37.726266,-122.479462
479,Locat

There are 271 SF stations out of 482 stations in the bay-wheels network in US

## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [11]:
# 37.802747 and longitude -122.405861
def build_request_body():
  request_body = {
      "origins": [],
      "destinations": [{
      "latitude": 37.8024,
      "longitude": -122.4058
      }],
      "travelMode": "Driving"
  }
  for row_series in df_sf.iterrows():
    request_body["origins"].append({
        "latitude": row_series[1]["latitude"],
        "longitude": row_series[1]["longitude"]
        })

  return request_body

response = requests.post("https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AjDIyFMNcKFfc6oWn8OmF_gxlT8rUMkepcqE6YB2z2EsGyyRFQMSAMXVoKNFH8Lq", json=build_request_body())
df_distance_bike_share = pd.json_normalize(response.json()["resourceSets"][0]["resources"][0]["results"])
df_distance_bike_share = df_distance_bike_share.join(df_sf.reset_index())

min_index = df_distance_bike_share["travelDistance"].idxmin()
df_distance_bike_share.loc[min_index]

destinationIndex                                                                         0
originIndex                                                                            218
totalWalkDuration                                                                        0
travelDistance                                                                       0.965
travelDuration                                                                      3.2333
index                                                                                  416
__type                                   Location:http://schemas.microsoft.com/search/l...
bbox                                     [37.798896982429326, -122.41975667826308, 37.8...
name                                     710 Lombard St, San Francisco, CA 94133, Unite...
confidence                                                                            High
entityType                                                                         Address

The closest staion is on Lombard St. at Columbus Avenue. I got a different answer than from lab A because this route api takes the actualy driving route into account so it's not just based on manhattan distance so we get a more realistic answer.

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.